In [4]:
#!pip install bs4
import pickle
import os
import datetime as dt
import pandas_datareader.data as web 
import bs4, requests
import pandas as pd
from bs4 import BeautifulSoup as bs4

In [2]:
def save_nifty50_tickers():
    res = requests.get('https://en.wikipedia.org/wiki/NIFTY_50')
    soup = bs4(res.text,'html.parser')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.find_all('tr')[1:]:
        ticker = row.find_all('td')[1].text
        tickers.append(ticker)
        
    with open("nifty50_tickers.pickle", "wb") as f:
        pickle.dump(tickers, f)
        
    return tickers
#url = 'https://en.wikipedia.org/wiki/NIFTY_50'
#res = requests.get(url)
#soup = bs4(res.text, 'html.parser')

In [5]:
#save_nifty50_tickers()

In [10]:
def get_data_from_yahoo(reload_nifty50=False):
    if reload_nifty50:
        tickers = save_nifty50_tickers()
    else:
        with open("nifty50_tickers.pickle", "rb") as f:
            tickers = pickle.load(f)
    
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')
    
    start = dt.datetime(2000,1,1)
    end = dt.datetime(2020,12,31)
    
    for ticker in tickers:
        if not os.path.exists(f'stock_dfs/{ticker}'):    
            df = web.DataReader(ticker, 'yahoo', start, end)
            df.to_csv(f'stock_dfs/{ticker}')
            print(f'{ticker}.csv created successfully!')
        else:
            print(f'{ticker}.csv already exists')

In [13]:
#get_data_from_yahoo()

In [23]:
def compile_data():
    with open("nifty50_tickers.pickle", "rb") as f:
        tickers = pickle.load(f)
    
    main_df = pd.DataFrame()
    for count,ticker in enumerate(tickers):
        df = pd.read_csv(f"stock_dfs/{ticker}")
        df.set_index('Date', inplace=True)
        
        df.rename(columns = {'Adj Close': ticker}, inplace=True)
        df.drop(['Open','Close','Volume','High','Low'], 1, inplace=True)
        
        if main_df.empty:
            main_df = df
        else:
            main_df = main_df.join(df, how='outer')
            
        if count%10==0:
            print(count)
    print(main_df.head())
    main_df.to_csv('stock_dfs/nifty50_joined_')

In [24]:
compile_data()

0
10
20
30
40
            ADANIPORTS.NS  ASIANPAINT.NS  AXISBANK.NS  BAJAJ-AUTO.NS  \
Date                                                                   
2000-01-03            NaN            NaN     0.976623            NaN   
2000-01-04            NaN            NaN     1.014331            NaN   
2000-01-05            NaN            NaN     0.987936            NaN   
2000-01-06            NaN            NaN     0.963426            NaN   
2000-01-07            NaN            NaN     0.916292            NaN   

            BAJFINANCE.NS  BAJAJFINSV.NS  BHARTIARTL.NS   BPCL.NS  \
Date                                                                
2000-01-03            NaN            NaN            NaN  0.431323   
2000-01-04            NaN            NaN            NaN  0.400264   
2000-01-05            NaN            NaN            NaN  0.388866   
2000-01-06            NaN            NaN            NaN  0.410852   
2000-01-07            NaN            NaN            NaN  0.410366  